In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
os.chdir("../")


In [2]:
data=pd.read_csv("./merged/len3_ship_mon.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["Approved"])
data

,year,month,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,year_month,stock_rx_lag,stock_otc_lag,stock_self_lag,Approved,KEGG,Ingredient,variable
36,2009,1,114,解熱鎮痛消炎剤,9028279,0,0.0,2390534,281726,0.0,2009-01,NaN,NaN,NaN,1985.0,D00141,Indometacin,id1
37,2009,1,114,解熱鎮痛消炎剤,9028279,0,0.0,2390534,281726,0.0,2009-01,NaN,NaN,NaN,1985.0,D00126,Ibuprofen,id1
38,2009,2,114,解熱鎮痛消炎剤,6285471,0,0.0,2842393,288192,0.0,2009-02,2390534,281726,0.0,1985.0,D00141,Indometacin,id1
39,2009,2,114,解熱鎮痛消炎剤,6285471,0,0.0,2842393,288192,0.0,2009-02,2390534,281726,0.0,1985.0,D00126,Ibuprofen,id1
40,2009,3,114,解熱鎮痛消炎剤,9085356,0,0.0,2664310,291364,0.0,2009-03,2842393,288192,0.0,1985.0,D00141,Indometacin,id1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40971,2023,4,629,その他の化学療法剤,5676488,-,0.0,17192413,-,0.0,2023-04,19302510,-,0.0,2011.0,D00282,Clotrimazole,id3
41084,2023,1,799,他に分類されない治療を主目的としない医薬品,1124555,234387,0.0,1283795,204783,0.0,2023-01,1104933,235072,0.0,2001.0,D03365,Nicotine,id1
41085,2023,2,799,他に分類されない治療を主目的としない医薬品,1299090,285720,0.0,1378071,167623,0.0,2023-02,1283795,204783,0.0,2001.0,D03365,Nicotine,id1
41086,2023,3,799,他に分類されない治療を主目的としない医薬品,1547593,577328,0.0,1497563,170752,0.0,2023-03,1378071,167623,0.0,2001.0,D03365,Nicotine,id1


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [3]:
data["elasped"]=data["year"]-data["Approved"]
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
# count ver.
# aggregated_data = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()

# dummy ver.
aggregated_df = pd.concat([
    data.groupby(['year', 'id',"month"]).first(), 
    dummies.groupby([data['year'],data['id'],data["month"]]).apply(lambda x: (x[colnames] > 0).any().astype(int))
], axis=1).reset_index()
aggregated_df

,year,id,month,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,...,elasped_31,elasped_32,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38,elasped_39,elasped_40
0,2009,114,1,解熱鎮痛消炎剤,9028279,0,0.0,2390534,281726,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2009,114,2,解熱鎮痛消炎剤,6285471,0,0.0,2842393,288192,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2009,114,3,解熱鎮痛消炎剤,9085356,0,0.0,2664310,291364,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2009,114,4,解熱鎮痛消炎剤,11595639,0,0.0,2686819,277389,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2009,114,5,解熱鎮痛消炎剤,8564391,0,0.0,2399578,289042,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,2023,629,4,その他の化学療法剤,5676488,-,0.0,17192413,-,0.0,...,0,0,0,0,0,0,0,0,0,0
6004,2023,799,1,他に分類されない治療を主目的としない医薬品,1124555,234387,0.0,1283795,204783,0.0,...,0,0,0,0,0,0,0,0,0,0
6005,2023,799,2,他に分類されない治療を主目的としない医薬品,1299090,285720,0.0,1378071,167623,0.0,...,0,0,0,0,0,0,0,0,0,0
6006,2023,799,3,他に分類されない治療を主目的としない医薬品,1547593,577328,0.0,1497563,170752,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
colnames

['elasped_m13',
 'elasped_m12',
 'elasped_m11',
 'elasped_m10',
 'elasped_m9',
 'elasped_m8',
 'elasped_m7',
 'elasped_m6',
 'elasped_m5',
 'elasped_m4',
 'elasped_m3',
 'elasped_m2',
 'elasped_m1',
 'elasped_0',
 'elasped_1',
 'elasped_2',
 'elasped_3',
 'elasped_4',
 'elasped_5',
 'elasped_6',
 'elasped_7',
 'elasped_8',
 'elasped_9',
 'elasped_10',
 'elasped_11',
 'elasped_12',
 'elasped_13',
 'elasped_14',
 'elasped_15',
 'elasped_16',
 'elasped_17',
 'elasped_18',
 'elasped_19',
 'elasped_20',
 'elasped_21',
 'elasped_22',
 'elasped_23',
 'elasped_24',
 'elasped_25',
 'elasped_26',
 'elasped_27',
 'elasped_28',
 'elasped_29',
 'elasped_30',
 'elasped_31',
 'elasped_32',
 'elasped_33',
 'elasped_34',
 'elasped_35',
 'elasped_36',
 'elasped_37',
 'elasped_38',
 'elasped_39',
 'elasped_40']

connect generic usage

In [5]:
dusage=pd.read_csv("./generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [6]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df

,year,id,month,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,...,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38,elasped_39,elasped_40,Year,Percentage
0,2009,114,1,解熱鎮痛消炎剤,9028279,0,0.0,2390534,281726,0.0,...,0,0,0,0,0,0,0,0,2009,35.8
1,2009,114,2,解熱鎮痛消炎剤,6285471,0,0.0,2842393,288192,0.0,...,0,0,0,0,0,0,0,0,2009,35.8
2,2009,114,3,解熱鎮痛消炎剤,9085356,0,0.0,2664310,291364,0.0,...,0,0,0,0,0,0,0,0,2009,35.8
3,2009,114,4,解熱鎮痛消炎剤,11595639,0,0.0,2686819,277389,0.0,...,0,0,0,0,0,0,0,0,2009,35.8
4,2009,114,5,解熱鎮痛消炎剤,8564391,0,0.0,2399578,289042,0.0,...,0,0,0,0,0,0,0,0,2009,35.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,2022,799,8,他に分類されない治療を主目的としない医薬品,1500193,305986,0.0,1739639,384543,0.0,...,0,0,0,0,0,0,0,0,2022,79.0
5868,2022,799,9,他に分類されない治療を主目的としない医薬品,1572585,436842,0.0,2150410,340113,0.0,...,0,0,0,0,0,0,0,0,2022,79.0
5869,2022,799,10,他に分類されない治療を主目的としない医薬品,1523822,445750,0.0,1253828,311956,0.0,...,0,0,0,0,0,0,0,0,2022,79.0
5870,2022,799,11,他に分類されない治療を主目的としない医薬品,1471887,210242,0.0,1204241,310745,0.0,...,0,0,0,0,0,0,0,0,2022,79.0


In [7]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,month,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,...,id_321,id_332,id_339,id_391,id_399,id_441,id_449,id_625,id_629,id_799
0,2009,114,1,解熱鎮痛消炎剤,9028279,0,0.0,2390534,281726,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2009,114,2,解熱鎮痛消炎剤,6285471,0,0.0,2842393,288192,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2009,114,3,解熱鎮痛消炎剤,9085356,0,0.0,2664310,291364,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2009,114,4,解熱鎮痛消炎剤,11595639,0,0.0,2686819,277389,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2009,114,5,解熱鎮痛消炎剤,8564391,0,0.0,2399578,289042,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,2022,799,8,他に分類されない治療を主目的としない医薬品,1500193,305986,0.0,1739639,384543,0.0,...,0,0,0,0,0,0,0,0,0,1
5868,2022,799,9,他に分類されない治療を主目的としない医薬品,1572585,436842,0.0,2150410,340113,0.0,...,0,0,0,0,0,0,0,0,0,1
5869,2022,799,10,他に分類されない治療を主目的としない医薬品,1523822,445750,0.0,1253828,311956,0.0,...,0,0,0,0,0,0,0,0,0,1
5870,2022,799,11,他に分類されない治療を主目的としない医薬品,1471887,210242,0.0,1204241,310745,0.0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
df_with_dummies.to_csv("./merged/len3_ship_mon_agg_drop_never.csv",encoding="shift-jis")